In [1]:
import os
os.chdir('..')

In [2]:
from latent_actions.data.dataset import EpisodicDataset
import pickle
import numpy as np
from panda_gym.envs.robots.panda import Panda
from panda_gym.pybullet import PyBullet

/Users/shivampatel/miniconda3/envs/la/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
pybullet build time: Aug 21 2022 17:44:27


## Old Data

In [3]:
with open('data/rpnp_traj_final.pkl', 'rb') as fp:
    epi_ds = pickle.load(fp)

In [4]:
sim = PyBullet(render=False)
robot = Panda(sim, block_gripper=False, base_position=np.array([0.0, 0.0, 0.0]), control_type='ee')

In [5]:
for epi in epi_ds.episodes:
    for st in epi.steps:
        robot.set_joint_angles(st.context['joint_angles'])
        st.context['ee_pos'] = robot.get_ee_position()

In [11]:
for epi in epi_ds.episodes:
    for i in range(len(epi.steps)-1):
        epi.steps[i].ee_velocity = epi.steps[i+1].context['ee_pos'] - epi.steps[i].context['ee_pos']
        epi.steps[i].gripper_velocity = epi.steps[i+1].context['gripper_width'] - epi.steps[i].context['gripper_width']

In [12]:
epi_ds.episodes[0].steps[0]

Step(joint_velocity=array([ 1.72166465e-04, -3.71862741e-04,  7.93984552e-04, -1.42910645e-05,
        2.06962128e-05, -5.42213092e-05,  1.01906034e-03]), ee_velocity=array([ 4.25061926e-06, -1.41474964e-06,  5.20902093e-06]), context={'joint_angles': array([ 0.15133288,  0.57108017, -0.1546331 , -2.21932278,  0.18590827,
        2.78850058,  0.77104332]), 'gripper_width': 0.08003650605678558, 'ee_pos': array([ 0.53945997, -0.01707561,  0.01404306])}, gripper_velocity=0.0, human_action=array([], dtype=float64))

In [13]:
for epi in epi_ds.episodes:
    epi.steps = epi.steps[:-1]
    for st in epi.steps:
        st.gripper_velocity = np.array([st.gripper_velocity])
        st.context['gripper_width'] = np.array([st.context['gripper_width']])

In [14]:
epi_ds.dump('data/rpnp_traj_with_pose_old.pkl')

## New Data

In [15]:
with open('data/rpnp_traj_pose_o1g1.pkl', 'rb') as fp:
    epi_ds1 = pickle.load(fp)
with open('data/rpnp_traj_pose_o1g2.pkl', 'rb') as fp:
    epi_ds2 = pickle.load(fp)
with open('data/rpnp_traj_pose_o2g1.pkl', 'rb') as fp:
    epi_ds3 = pickle.load(fp)
with open('data/rpnp_traj_pose_o2g2.pkl', 'rb') as fp:
    epi_ds4 = pickle.load(fp)

In [22]:
epi_ds = EpisodicDataset()
epi_ds.episodes = epi_ds1.episodes + epi_ds2.episodes + epi_ds3.episodes + epi_ds4.episodes

In [25]:
for epi in epi_ds.episodes:
    for st in epi.steps:
        st.context['gripper_width'] = min(max(st.context['gripper_width'], 0), 0.08)

In [26]:
for epi in epi_ds.episodes:
    for i in range(len(epi.steps)-1):
        epi.steps[i].ee_velocity = epi.steps[i+1].context['pose'].translation - epi.steps[i].context['pose'].translation
        epi.steps[i].gripper_velocity = epi.steps[i+1].context['gripper_width'] - epi.steps[i].context['gripper_width']

In [27]:
epi_ds.episodes[0].steps[0]

Step(joint_velocity=array([-0.00018434, -0.00188449,  0.00042627,  0.00015077, -0.00033054,
        0.00025315,  0.00034416]), ee_velocity=array([8.00519558e-07, 1.02541019e-06, 1.18646757e-06]), context={'joint_angles': array([ 2.65130738e-03,  6.91122496e-01, -3.48891762e-02, -2.71925582e+00,
        9.43209190e-03,  3.43267063e+00,  7.85066110e-01]), 'gripper_width': 0.08, 'pose': RigidTransform(rotation=np.array([[ 0.99920478, -0.03298786,  0.02196263],
       [-0.03242571, -0.99913973, -0.02547802],
       [ 0.0227842 ,  0.0247456 , -0.9994341 ]]), translation=np.array([ 0.33498167, -0.01859357, -0.07637264]), from_frame='franka_tool', to_frame='world')}, gripper_velocity=0.0, human_action=array([], dtype=float64))

In [34]:
for epi in epi_ds.episodes:
    epi.steps = epi.steps[:-1]
    for st in epi.steps:
        st.gripper_velocity = np.array([st.gripper_velocity])
        st.context['gripper_width'] = np.array([st.context['gripper_width']])
        st.context['ee_pos'] = st.context['pose'].translation
        st.context['ee_rot'] = st.context['pose'].rotation

(array([ 0.34209884, -0.01871496, -0.07844445]),
 array([ 0.34202186, -0.01870173, -0.07684636]))